In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2


# Base network to be shared (reused)
def create_base_network(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, pooling='avg')
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.Dense(128, activation='relu')(x)
    return models.Model(base_model.input, x)

